전이학습

- 큰 데이터 셋을 사용하여 훈련된 모델의 가중치를 사용

- 특정 대상을 예측하도록 훈련된 모델을 다른 대상에 사용하는 것.

- 사전 훈련된 모델을 갖고 와서 마지막 완전 연결층 부분만 새로 생성.

- 작은 데이터셋 학습시 과대 적합 예방.
- 학습 빠르게 수행.

#### 전이 학습 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

from torchinfo import summary                                           # 모델 볼 수 있음.
from torchmetrics.functional.classification import multiclass_accuracy

In [6]:
### 데이터 정보
img_dir = '../data/image'

# Resnet 전처리 
# resize+size = [256] interpolation = InterpolationMode.BILINEAR
# central crop of crop_size = [224]
# first rescaled to [0.0, 1.0] and then
# nomalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]
preprocessing = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 이미지 데이터셋 생성
imgDS = ImageFolder(root=img_dir, transform = preprocessing)

print(imgDS.classes, imgDS.targets, imgDS.imgs, sep='\n')

['afolder', 'bfolder']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[('../data/image\\afolder\\angry\\0AvKtuzA7LfxnKaO0bey9mQMLnxXad73.jpg', 0), ('../data/image\\afolder\\angry\\0C5yo7GxMy8lztxNZvSdfEx2gSPRTR701.jpg', 0), ('../data/image\\afolder\\angry\\0C9jEgFQHsh36W5U2u5CA98lB7C5eX806.jpg', 0), ('../data/image\\afolder\\angry\\0RXraPIKC00Dz1qkuMbj8XbuR80g5Z893.jpg', 0), ('../data/image\\afolder\\angry\\0TSpHUDh1xIQthYrtH8E1UZQ72rcTZ68.jpg', 0), ('../data/image\\afolder\\angry\\0aNyXBrmNA7XdefwHvgO2n1rnpqQAp885.jpg', 0), ('../data/image\\afolder\\

In [7]:
# 데이터 로더 생성
imgDL = DataLoader(imgDS, batch_size=3, shuffle = True, drop_last=True)
for (img, label) in imgDL :
    print(img.shape, label)

torch.Size([3, 3, 224, 224]) tensor([1, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 0])
torch.Size([3

In [9]:
# 모델 설계 및 설정

# 사전 학습된 모델 인스턴스 생성
res_model = resnet18(weights=ResNet18_Weights.DEFAULT)

# 전결합층 변경
# in_features : FeatureMap에서 받은 피처 수, Out_features : 출력/분류 클래스 수
res_model.fc=nn.Linear(in_features=512, out_features=3)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\KDP-48/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


In [10]:
summary(res_model, (3, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [3, 3]                    --
├─Conv2d: 1-1                            [3, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [3, 64, 112, 112]         128
├─ReLU: 1-3                              [3, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [3, 64, 56, 56]           --
├─Sequential: 1-5                        [3, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [3, 64, 56, 56]           --
│

In [19]:
## ResNet18 Feature Module 파라미터 requires_grad = True ==> False 비활성화
for name, param in res_model.named_parameters() :
    print(name, param.requires_grad, end='             ====>  ')
    param.requires_grad = False
    print(param.requires_grad)

# ResNet18 Full Connected Module 파라미터 requires_grad = False ===> True 활성화
for name, param in res_model.fc.named_parameters():
    print(name, param.requires_grad, end="          ====>  ")
    param.requires_grad= True
    print(param.requires_grad)


conv1.weight False             ====>  False
bn1.weight False             ====>  False
bn1.bias False             ====>  False
layer1.0.conv1.weight False             ====>  False
layer1.0.bn1.weight False             ====>  False
layer1.0.bn1.bias False             ====>  False
layer1.0.conv2.weight False             ====>  False
layer1.0.bn2.weight False             ====>  False
layer1.0.bn2.bias False             ====>  False
layer1.1.conv1.weight False             ====>  False
layer1.1.bn1.weight False             ====>  False
layer1.1.bn1.bias False             ====>  False
layer1.1.conv2.weight False             ====>  False
layer1.1.bn2.weight False             ====>  False
layer1.1.bn2.bias False             ====>  False
layer2.0.conv1.weight False             ====>  False
layer2.0.bn1.weight False             ====>  False
layer2.0.bn1.bias False             ====>  False
layer2.0.conv2.weight False             ====>  False
layer2.0.bn2.weight False             ====>  False
layer

In [15]:
# 학습 준비
optimizer = optim.Adam(res_model.fc.parameters())
loss_fn = nn.CrossEntropyLoss()                     # 손실함수 정의
EPOCHS = 3

In [22]:
# 모델 설계 및 설정

# 사전 학습된 모델 인스턴스 생성
res_model = resnet18(weights=ResNet18_Weights.DEFAULT)

# 전결합층 변경
### in_features : FeatureMap에서 받은 피처 수, out_features : 출력/ 분류 클래스 수
res_model.fc = nn.Linear(in_features = 512, out_features=3)